# [Regression Trees (Decision Tree for regression problems)](https://youtu.be/RANHxyAvtM4?list=PLKnIA16_Rmvbr7zKYQuBfsVkjoLcJgxHH)

### Why do we need Decision Tree for regression when we already have linear Regression?  
Well in most of the cases linear regression does the job for us... still regression tree is preferred to use when data is non linear or when you have to do a lot of preprocessing.  
<br>
When its a classification problem... we calculate the information gain by taking all columns as root one by one and finally choose the one which returns highest information gain, we repeat this recursively until we reach the leaf node (if we let the algo complete that is).  <br>  
Now we can't calculate information gain because we can't calculate neither entropy nor gini impurity because the label column is not categorical type... its numerical...what to do? ofc: use regression tree.
<br>

---

### Working.  
main thing to understand here is how to make splits and how to choose the optimal split on each step. <br><br>

![ytss](assets/17_regressionTree.png)   
The above example is of the students who got different marks based on hours that they studied. 
<br>

1. Take first two points and make a split on their mean. splitting the tree into two... one side only point on the other all of the other points.
2. Having the tree splitted, now if there comes a new student whose number of hours fall into left side of first split, model says its marks are equal to first data point. and if the number of hours that new entered student studied fall into the right side of split... model says that numbers = mean of all the student's marks on the right side of tree (the horizontal lines in the image are means of marks calculated on different splits.)
3. next step is to calculated residual error, that is the squared sum of distance of each point from the mean of marks calculated on any particular split. After caculating SSE(Squared Sum of Errors) we plot it on a grah, where x-axis has marks'mean (values we use to split the tree) and y-axis has error (value of SSE calculated on each split ), upto this point first split is successfully complete.
4. Now we take 2nd and 3rd data point, caculate their mean, split the data their to make tree. calculate SSE on both sides. add them up and plot it on the next graph.
5. Do the step 4 repetitively and plot the SSE for evey split on the grpah, and when its completed we then choose the split where SSE is minimum (we cannow easily tell from the graph). at this point we now know where the first split will occur. we take that and split the tree into two and then repeat this process for evey child on both sides and determine their splits until tree reaches the leaf nodes (meaning there is only one data point on both sides of new splitted node)

#### Now as it is now... the tree doesn't stop until each data point is splitted into separate leaf node... we don't want that... we want clusters separated from each other not data points.  

There are many way to do that. For now let's say that if any splitted side of node has less than n number of data points. we won't split any further.
<br><br>
if a node has less than 4 data points on its let's say left side and a new entered test data point comes and falls into this cluster, we will just calculate the mean of those data points and return that as predicted value for the new data point.  


---

## upto this point we just had 2D data, what if we had more dimensions?🤔  
<br>

![ytss](assets/18_regressionTree3D.png)  
<br>

Now that we have 3D data... what do we do? it ain't quantum science actually. just plot a scatter plot of every columns with label column... in our example's case. hour vs marks and cgpa vs marks. and boom that gives us the good old 2d regression tree problem. the next questionn comes... how to split now? because now we have 2 SSEs at the same split. well the answer is that we take whichever one is smaller. So we split it on the deature which returns smaller value of SSE, in here if it gets splitted on hours the plane that splits the tree will be parallel to hours and if it gets splitted on cgpa the place splitting the tree will be parallel to cgpa.


---

## Code


In [7]:
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_california_housing
import numpy as np
# from pandas_datareader import data

In [12]:
data = fetch_california_housing()
df = pd.DataFrame(data.data)
df.columns = data.feature_names
df['MEDV'] = data.target
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MEDV
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [22]:
df.shape

(20640, 9)

In [16]:
X = df.iloc[:,0:-1]
y = df.iloc[:,-1]

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [36]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
r2_score(y_test,lr.predict(X_test))

# just for refrence

0.575787706032451

In [71]:
rt = DecisionTreeRegressor(max_depth=9)

In [72]:
rt.fit(X_train,y_train)

,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",9
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_le

In [73]:
y_pred = rt.predict(X_test)
r2_score(y_test,y_pred)

0.681844324593396

In [ ]:
rd = DecisionTreeRegressor()

In [83]:
param_grid = {
    'criterion':["squared_error", "friedman_mse", "absolute_error"],
    'max_depth':[2,4,9,10,None],
    'max_features':[0.25,0.5,1.0],
    'min_samples_split':[0.25,0.5,1.0]
}

grid = GridSearchCV(DecisionTreeRegressor(),param_grid=param_grid)
grid.fit(X_train,y_train)
grid.best_score_,grid.best_params_

(np.float64(0.46149607986708385),
 {'criterion': 'friedman_mse',
  'max_depth': 10,
  'max_features': 1.0,
  'min_samples_split': 0.25})

In [85]:
for importance, nam in sorted(zip(rt.feature_importances_,X_train.columns),reverse=True):
    print(nam, " ", importance)

MedInc   0.6312959897155721
AveOccup   0.13177578511227686
Latitude   0.07415639759676722
Longitude   0.060183447063199356
HouseAge   0.04382743959994155
AveRooms   0.03652210723577012
Population   0.01131499803722416
AveBedrms   0.010923835639248545
